In [23]:
from sentence_transformers import CrossEncoder
from opensearch_interface import OpenSearchClient
from typing import List, Union
import numpy as np
from loguru import logger

In [24]:

class ReRanker:
    '''
    Cross-Encoder models achieve higher performance than Bi-Encoders, 
    however, they do not scale well to large datasets. The lack of scalability
    is due to the underlying cross-attention mechanism, which is computationally
    expensive.  Thus a Bi-Encoder is best used for 1st-stage document retrieval and 
    a Cross-Encoder is used to re-rank the retrieved documents. 

    https://www.sbert.net/examples/applications/cross-encoder/README.html
    '''

    def __init__(self, model_name: str='cross-encoder/ms-marco-MiniLM-L-6-v2', local_files: bool=False):
        self.model_name = model_name
        self.model = CrossEncoder(self.model_name, automodel_args={'local_files_only':local_files})
        self.score_field = 'cross_score'

    def _cross_encoder_score(self, 
                             results: List[dict], 
                             query: str, 
                             cross_score_key: str='cross-score',
                             return_scores: bool=False
                             ) -> Union[np.array, None]:
        '''
        Given a list of hits from a Retriever:
            1. Scores hits by passing query and results through CrossEncoder model. 
            2. Adds cross-score key to hits dictionary. 
            3. If desired returns np.array of Cross Encoder scores.

        Args
        ----
        results: List[dict]
            List of search results from OpenSearch client.
        query: str
            User query.
        cross_score_key: str='cross-score'
            Name of key/field that the new calculated cross encoder score will be associated with.
        return_scores: bool=False
            If True, returns a np.array of cross encoder scores. 

        Returns
        -------
        Either returns a np.array of cross encoder scores if "return_scores" is True, otherwise
        nothing is returned.  The primary purpose of this function is to update the "results" dict. 
        '''
        
        #build query/content list 
        #create a list of lists that contains the query and the content field of each result from "results"
        #important the cross_input variable must be a list of lists
        cross_input = None
        
        #get scores
        #call self.model to get predicted scores on the cross_input
        cross_scores = None

        #enumerate through the results and update each dict with the cross_score_key arg as key and value as the new score:
        #Example:
             # {'cross-score' : 5.12345}
        for i, result in enumerate(results):
            None

        if return_scores:
            return cross_scores

    def rerank(self, results: List[dict], query: str, top_k: int=10, threshold: float=None) -> List[dict]:
        '''
        Given a list of search results from OpenSearch client, results are scored with a Cross Encoder 
        and returned in sorted order by the cross_score field.  Threshold allows user to filter out 
        results that do not meet cross_score threshold value:

        Args
        ----
        results: List[dict]
            List of search results from OpenSearch client.
        query: str
            User query.
        top_k: int=10
            Number of reranked results to return
        threshold: float=None
            If None, top_k results will be returned.  
            If float value is present, only results with a cross_score that meet or exceed the threshold
            will be retuned.  This arg is present to prevent very low scoring document from being returned. 

        Returns
        -------
        List of reranked search results. 
        '''
        # call the internal _cross_encoder_score function (it's ok that nothing is returned here)
        # the results dictionary is being updated 
        None

        #sort results by the new cross-score field
        sorted_hits = None

        #if user wants to set a threshold we need to account for that
        if threshold or threshold == 0:

            #filter sorted_hits by the threshold value
            filtered_hits = None
            
            if not any(filtered_hits):
                logger.warning(f'No hits above threshold {threshold}. Returning top {top_k} hits.')
                return sorted_hits[:top_k]
            return filtered_hits
            
        #if no threshold was set return top_k sorted_hits
        return None

### Instantiate the ReRanker instance

In [12]:
ranker = ReRanker()

In [32]:
#set index name
index_name = 'impact-theory-minilm-196'

In [33]:
#set query
query = "how do I change my life for good"

### Test ReRanker class by conducting a hybrid search

In [35]:
#create new OpenSearch client
osclient = OpenSearchClient('sentence-transformers/all-MiniLM-L6-v2')

In [36]:
#conduct hybrid search
response = osclient.hybrid_search(query, index_name, index_name, kw_size=50, vec_size=50)

In [37]:
#ranked the hybrid response
ranked = ranker.rerank(response, query, top_k=10)

Bad pipe message: %s [b'L;\xcb\xa9h\xbc>\n,\xc1\xdd\xb2JV\n\xe7\xc1[ \nT\x8a\x8bl\x8e=\x04']
Bad pipe message: %s [b'\xc82_B\x9f\xd5bya\xe6%\xd6\xba\xd3\x19\x8d\xb7\x89 X\x8f\xfd\xc8\xfe\xb4\xdb\xa4\x0e\xba\xcd(D\xdaoP\xdb\xd5\xfb\r\x95\xec\xf3\xba\xdb\x89\xec\x8bJ\x08\x15x\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t1']
Bad pipe message: %s [b'.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04', b'\x03\x06', b'\x07\x08']
Bad pipe message: %s [b'\t\x08\n\x08\x0b\x08\x04']
Bad pipe message: %s [b'\x08\x06\x04\x01\x05\x01\x06', b'']
Bad pipe message: %s [b"\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 0\x96o\x04\x1eb'\xb1q\xe4\x80\xe1\xef\xdey\xfc\xab\xcf_k$\x0c"]
Bad pipe message: %s [b'|>td{\xb9o\xdcm\xbe\xdb\x02\xa3\xd0\xc5\xbe\x82%\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0', 